In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from copy import deepcopy

from boltzmann.data.mnist import load_mnist
from boltzmann.restricted.base import train
from boltzmann.restricted.bernoulli.common import HintonInitializer
from boltzmann.restricted.bernoulli.dense import (
    DenseBernoulliRBM, initialize_fantasy_latent, enlarge_latent,
    get_reconstruction_error)
from boltzmann.utils import History, ExponentialMovingAverage

In [ ]:
# Global parameters

IMAGE_SIZE = (16, 16)
LATENT_SIZE = 64
BATCH_SIZE = 128
SEED = 42

INCREMENT = 8

tf.random.set_seed(SEED)

In [ ]:
# Data
(X, y), _ = load_mnist(image_size=IMAGE_SIZE, binarize=True, minval=0, maxval=1)

In [ ]:
def build_and_train(epochs: int, cache_path: str = None):
    ambient_size = IMAGE_SIZE[0] * IMAGE_SIZE[1]
    rbm = DenseBernoulliRBM(ambient_size, LATENT_SIZE, HintonInitializer(X, seed=SEED))
    if cache_path is None:
        dataset = tf.data.Dataset.from_tensor_slices(X)
        epochs = 20
        # epochs = 1  # XXX: test!
        dataset = dataset.shuffle(10000, seed=SEED).repeat(epochs).batch(BATCH_SIZE)
        fantasy_latent = initialize_fantasy_latent(rbm.latent_size, BATCH_SIZE, seed=SEED)
        optimizer = tf.optimizers.Adam()
        fantasy_latent = train(rbm, optimizer, dataset, fantasy_latent)
    else:
        try:
            with open(cache_path, 'rb') as f:
                U, bv, bh, fantasy_latent = pickle.load(f)
            rbm.kernel.assign(U)
            rbm.ambient_bias.assign(bv)
            rbm.latent_bias.assign(bh)
            fantasy_latent = tf.constant(fantasy_latent)
        except FileNotFoundError as e:
            print(f'[WARNING]: Cannot find file "{cache_path}", create new file on that path.')
            rbm, fantasy_latent = build_and_train(epochs, cache_path=None)
        with open(cache_path, 'wb') as f:
            U = rbm.kernel.numpy()
            bv = rbm.ambient_bias.numpy()
            bh = rbm.latent_bias.numpy()
            pickle.dump((U, bv, bh, fantasy_latent.numpy()), f)
    return rbm, fantasy_latent

In [ ]:
base_rbm, base_fantasy_latent = build_and_train(1, cache_path='../dat/base_rbm_for_rg_flow.pkl')

In [ ]:
get_reconstruction_error(base_rbm, X[:1000])

In [ ]:
# Initialize
history = History()
rbm = deepcopy(base_rbm)
fantasy_latent = deepcopy(base_fantasy_latent)
iter_step = 0

def log(iter_step):
    history.log(iter_step, 'kernel', rbm.kernel.numpy())
    history.log(iter_step, 'ambient_bias', rbm.ambient_bias.numpy())
    history.log(iter_step, 'latent_bias', rbm.latent_bias.numpy())

In [ ]:
# infinite loop of incrementing
while True:
    print('\n')
    print(f'The {iter_step + 1}th interation......')
    dataset = tf.data.Dataset.from_tensor_slices(X)
    epochs = 10  # enough epochs for ensuring the convergence of training.
    # epochs = 1  # XXX: test!
    dataset = dataset.shuffle(10000, seed=SEED).repeat(epochs).batch(BATCH_SIZE)
    inc_rbm, inc_fantasy_latent = enlarge_latent(rbm, fantasy_latent, INCREMENT)
    optimizer = tf.optimizers.Adam()
    inc_fantasy_latent = train(inc_rbm, optimizer, dataset, inc_fantasy_latent)

    rbm, fantasy_latent, iter_step = inc_rbm, inc_fantasy_latent, iter_step + 1

    log(iter_step)

In [ ]:
print('Current latent size:', rbm.latent_size)

In [ ]:
steps = sorted(list(history.logs.keys()))
kernel_diff_hist = []
for i, j in zip(steps[:-1], steps[1:]):
    U_i = history.logs[i]['kernel'][:, :LATENT_SIZE]
    U_j = history.logs[j]['kernel'][:, :LATENT_SIZE]
    kernel_diff_hist.append(U_j - U_i)
kernel_diff_hist = np.stack(kernel_diff_hist, axis=0)
kernel_diff_hist = ExponentialMovingAverage(0.9)(kernel_diff_hist, axis=0).numpy()

plt.plot(steps[1:], np.zeros_like(steps[1:]), '--', label='zero')

def plot_confidence_region(confidence, **plot_kwargs):
    lower = [np.quantile(x.reshape([-1]), (1 - confidence) / 2) for x in kernel_diff_hist]
    upper = [np.quantile(x.reshape([-1]), 1 - (1 - confidence) / 2) for x in kernel_diff_hist]
    plt.fill_between(steps[1:], lower, upper,
                     label=f'{(confidence * 100):.2f}% confidence region',
                     **plot_kwargs)

plot_confidence_region(0.6827, alpha=0.5)
plot_confidence_region(0.9544, alpha=0.25)
plot_confidence_region(0.9973, alpha=0.25)

plt.title('Averaged kernel difference history')
plt.legend(loc='lower right')
plt.show()